# Biblioteca Pandas

In [ ]:
# importar biblioteca
import pandas as pd

In [ ]:
local_arquivo = r'arquivos\petr4.csv'


Lendo arquivo csv local

In [ ]:
df = pd.read_csv(local_arquivo, decimal=',')
df

Tipo de objeto

In [ ]:
print(type(df))

Tipos de objetos dentro do dataframe

In [ ]:
df.dtypes

Descrever o formato dos dados (linhas, colunas)

In [ ]:
print(df.shape)

## Acesso à tabelas em html

datasets damodaran: http://pages.stern.nyu.edu/~adamodar/New_Home_Page/dataarchived.html

In [ ]:
# TABELAS DA PÁGINA DO DAMODARAN
link = 'http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html'

df_html = pd.read_html(link)
df_html

Tipo do objeto

In [ ]:
print(type(df_html))

## Tratando dados

Selecionando primeiro item da lista

In [ ]:
df_html = df_html[0]
df_html

Trocando o cabeçalho da coluna

In [ ]:
df_html.columns = df_html.loc[1]
df_html

Deletando colunas

In [ ]:
df_html.drop([0, 1], inplace=True)
df_html

In [ ]:
print(df_html.dtypes)

Substituindo strings dentro do dataframe

In [ ]:
df_html.replace('%', '', regex=True, inplace=True)
df_html.replace(',', '', regex=True, inplace=True)
df_html.replace('\$ ', '', regex=True, inplace=True)

Transformando objetos em dados numéricos

In [ ]:
df_html = df_html.apply(pd.to_numeric)

In [ ]:
df_html.dtypes

Agora podemos fazer operações com o dataframe

In [ ]:
df_html['S&P 500 (includes dividends)'] = df_html['S&P 500 (includes dividends)']/100

In [ ]:
df_html

Transformar o "Year" em index

In [ ]:
df_html.set_index('Year', inplace=True)
df_html

Ordenar pelo índice

In [ ]:
df_html.sort_index(ascending=False, inplace=True)
df_html

Ordernar por outras colunas

In [ ]:
df_html.sort_values(by='S&P 500 (includes dividends)', ascending=False, inplace=True)
df_html

## Código completo read_html

In [ ]:
# TABELAS DA PÁGINA DO DAMODARAN
link = 'http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/histretSP.html'

# Lê a tabela e já seleciona o primeiro item da lista de dataframes
df_html = pd.read_html(link)[0]

# Troca os títulos das colunas para a segunda linha do dataframe
df_html.columns = df_html.loc[1]

# Apaga a linha 0 e 1 do dataframe pois são iguais aos títulos
df_html.drop([0, 1], inplace=True)

# Substituindo strings que atrapalham a identificação correta dos tipos de objeto
df_html.replace('%', '', regex=True, inplace=True)
df_html.replace(',', '', regex=True, inplace=True)
df_html.replace('\$ ', '', regex=True, inplace=True)

# Transformando em objetos do tipo numérico
df_html = df_html.apply(pd.to_numeric)

# Dividindo a coluna da rentabilidade por 100 para ficar em porcentagem
df_html['S&P 500 (includes dividends)'] = df_html['S&P 500 (includes dividends)']/100

# Usando a data como índice
df_html['Year'] = pd.to_datetime(df_html['Year'], format='%Y')
df_html.set_index('Year', inplace=True)

df_html

## Trabalhando com datetime

In [ ]:
arquivo = r'arquivos\ibov.csv'
nome_papel = arquivo.split('\\')[-1][:-4]
nome_papel = nome_papel.upper()
print(nome_papel)
df = pd.read_csv(arquivo, decimal=',', thousands='.')
df

Apagando as colunas que não serão usadas

In [ ]:
df.drop(columns=['Hora', 'ULT'], inplace=True)

Transformar a data em datetime

In [ ]:
df.dtypes

In [ ]:
df['Data_teste'] = pd.to_datetime(df['Data'])
df.dtypes

Conferindo formato da data

aaaa-mm-dd

In [ ]:
df.sort_values(by='Data_teste', ascending=False)

In [ ]:
df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
df.sort_values(by='Data', ascending=False, inplace=True)

In [ ]:
df.drop(columns='Data_teste', inplace=True)
df

Usando a data como índice

In [ ]:
df.set_index('Data', inplace=True)

Transformando em dados mensais

In [ ]:
df_mensal = df.resample('M').last()
df_mensal

In [ ]:
df.rename(columns={'Fechamento': nome_papel}, inplace=True)
df

In [ ]:
df_anual = df.resample('Y').last()
df_anual

Calcular o retorno anual

In [ ]:
df_anual[f'retorno_{nome_papel}'] = df_anual[nome_papel].pct_change()
df_anual

## Juntando dois dataframes

In [ ]:
df_html

In [ ]:
df_sp = df_html.copy()
df_sp

Selecionando as primeiras colunas

In [ ]:
df_sp = df_sp[df_sp.columns[0:3]]
df_sp

Transformar a data e ano em apenas ano

In [ ]:
df_sp.index = df_sp.index.year
df_sp

In [ ]:
df_anual.index = df_anual.index.year
df_anual

In [ ]:
df_junto = df_anual.merge(df_sp, right_index=True, left_index=True)
df_junto.dropna(inplace=True)
df_junto = df_junto[['retorno_IBOV', 'S&P 500 (includes dividends)']]
df_junto

In [ ]:
df_junto.plot()

In [ ]:
# Retornos do IBOV
link_ibov = 'http://bvmf.bmfbovespa.com.br/indices/ResumoVariacaoAnual.aspx?Indice=IBOV&idioma=pt-br'
df_online = pd.read_html(link_ibov, decimal=',', thousands='.')[0].dropna()

# Limpando o dataframe
df_online.replace('\*', '', regex=True, inplace=True)
df_online.replace(' \(\)', '', regex=True, inplace=True)

# Transformando o ano em índice
df_online['Ano'] = pd.to_datetime(df_online['Ano'], format='%Y')
df_online.set_index('Ano', inplace=True)
df_online.index = df_online.index.year

# Transformando em %
df_online[['Variação anual em US$', 'Variação Anual Nominal']] = df_online[['Variação anual em US$', 'Variação Anual Nominal']]/100

df_online

Juntando o novo dataframe

In [ ]:
df_junto = df_junto.merge(df_online[['Variação anual em US$']], right_index=True, left_index=True)
df_junto

Plotando gráfico

In [ ]:
df_junto[['S&P 500 (includes dividends)', 'Variação anual em US$']].plot()

Descrevendo os dados

In [ ]:
df_junto.describe()

## Criando a função + Código completo retorno

In [ ]:
def dados_retornos(arquivo, freq='Y'):
    
    # Nome do ativo do arquivo
    nome_papel = arquivo.split('\\')[-1][:-4]
    nome_papel = nome_papel.upper()
    
    # Ler o arquivo em csv
    df = pd.read_csv(arquivo, decimal=',', thousands='.')
    
    # Dropando colunas não usadas
    df.drop(columns=['Hora', 'ULT', 'Abertura', 'Maxima', 'Minima'], inplace=True)
    
    # Trocando o nome 'Fechamento' para o nome do ativo
    df.rename(columns={'Fechamento': nome_papel}, inplace=True)

    # Transformando a data no formato correto de datetime
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    
    # Usando a coluna Data como índice
    df.set_index('Data', inplace=True)
    
    # Frequência dos dados 'M', 'Q', 'Y'
    df = df.resample(freq).last()
    
    # Ordenando por data menor para maior
    df.sort_index(ascending=True, inplace=True)
    
    # Calculando o retorno no período
    df[f'retorno_{nome_papel}'] = df[nome_papel].pct_change()
    
    # Apagando linhas onde existem dados em branco
    df.dropna(inplace=True)
    
    return df

In [ ]:
arquivo = r'arquivos\ibov.csv'
retorno_papel = dados_retornos(arquivo, 'Y')
retorno_papel

Juntando todos arquivos baixados

In [ ]:
# Pacote que faz iteração com arquivos e pastas
import glob

# Local onde se encontram os arquivos \ tipo do arquivo
path = 'arquivos'
todos_arquivos = glob.glob(path+'\*.csv')

# Criar um dataframe em branco
geral = pd.DataFrame()

# Loop executando nossa função e adicionando ao dataframe em branco 
for local in todos_arquivos:
    
    print(local)
    
    df = dados_retornos(local, 'M')
    geral = pd.concat([geral, df], axis=1)

geral

Apagando as linhas onde há dados em branco

In [ ]:
geral.dropna(inplace=True)
geral

Separando listas de colunas do tipo retorno e fechamento

In [ ]:
colunas_retorno = [coluna for coluna in geral.columns if 'retorno' in coluna]
colunas_fechamento = [coluna for coluna in geral.columns if 'retorno' not in coluna]
colunas_retorno

In [ ]:
geral[colunas_fechamento].drop(columns=['IBOV']).plot()

In [ ]:
geral[colunas_retorno].describe()